In [1]:
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from qiskit.primitives import StatevectorSampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

import matplotlib.pyplot as plt
# Import necessary Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import numpy as np


In [2]:
def controlled_rotation_on_n(theta, state):
    """
    Create a custom gate that applies a controlled Y-rotation (CRY)
    on a target qubit if the control qubits are in the specified state.

    Parameters:
    - theta: rotation angle for the Y-rotation.
    - state: list of bits representing the specific state to check for.
             Example: [1, 0, 1] means check for the |101> state.
    
    Returns:
    - A custom gate for detecting the specific state and applying the rotation.

    This code has been generated by OpenAI(2023) ChatGPT (Oct. 24 version).
    """
    n_controls = len(state)
    num_qubits = n_controls + 2  # n control qubits + 1 target qubit + 1 ancilla
    qc = QuantumCircuit(num_qubits, name=f"CRY_{''.join(map(str, state))}")  # Create the circuit

    # Prepare the control logic: flip control qubits as needed to detect the state
    for i, bit in enumerate(state):
        if bit == 0:
            qc.x(i)  # Apply X gate to qubits that should be |0> to detect |1>

    # Apply a multi-controlled Toffoli gate to the ancilla (last qubit)
    if n_controls == 1:
        qc.cx(0, n_controls + 1)  # Controlled-X for single qubit control
    elif n_controls == 2:
        qc.ccx(0, 1, n_controls + 1)  # Toffoli gate for 2 controls
    else:
        qc.mcx(list(range(n_controls)), n_controls + 1)  # General multi-controlled gate for n controls

    # Apply the controlled rotation (CRY) based on the ancilla qubit
    qc.cry(theta, n_controls + 1, n_controls)

    # Undo the control logic (reset the ancilla and control qubits)
    if n_controls == 1:
        qc.cx(0, n_controls + 1)
    elif n_controls == 2:
        qc.ccx(0, 1, n_controls + 1)
    else:
        qc.mcx(list(range(n_controls)), n_controls + 1)

    # Undo the X gates on the control qubits
    for i, bit in enumerate(state):
        if bit == 0:
            qc.x(i)

    # Convert to a gate
    custom_gate = qc.to_gate()
    return custom_gate

def limited_split_qc(qc, values: list, control, qbit_no:int, ancilla_no:int, limit: int):
    """
    Create a circuit that recursivelly split up to a certain number of qubits.
    """
    if qbit_no > limit:
        return 0
    if len(values)<=1:
        return 0
    
    mid = len(values) // 2
    left = values[:mid]
    right = values[mid:]
    if(sum(values)>0):
        f = sum(left)/sum(values)
    else:
        f = 0
    theta = 2*math.acos(math.sqrt(f))
    #print("rotate i_th qubits " + str(qbit_no)+ " of " + str(theta) + " controlled by " + str(control) + ".")
    binary_list_c_state = [int(bit) for bit in control]
    if(control == ""):
        qc.ry(theta, qbit_no)
    else:
        cry_gate = controlled_rotation_on_n(theta, binary_list_c_state)
        qbits = []
        qbits.extend(list(range(qbit_no)))
        qbits.append(qbit_no)
        qbits.append(ancilla_no)
        qc.append(cry_gate, qbits)
    limited_split_qc(qc, left, control + "0", qbit_no+1, ancilla_no, limit)
    limited_split_qc(qc, right, control + "1", qbit_no+1, ancilla_no, limit)

def gaussian_pmf(x, R, M=0):
    """
    discrete gaussian probability mass function with std proportionnal to R and mean M (default 0)
    """
    return np.exp(-np.pi * ((x-M)/R)**2)
    
def getProbValue(n_qubits: int, f, R):
    """
    Create a list of probatility value of a discrete gaussian distribution for a given range of integer value discretization.
    """
    lower = math.pow(2, n_qubits) / 2 * (-1)
    upper = math.pow(2, n_qubits) / 2
    probValue = [f(x, R) for x in range(int(lower), int(upper))] 
    return probValue

def split_qc(qc, values: list, control, qbit_no:int, ancilla_no:int):
    """
    Create a circuit that recursively split a state into a superposition of 2 states with mass define as the sum of each of the 2 half of the values list.
    This will result in a superposition of state with weight matching the values list.
    Use the procedure described by Grover and Rudolph here (https://arxiv.org/abs/quant-ph/0208112)

    Here we compute the sum over the value of the array, but this method would also work with only the probability function as parameter (as long as it is efficiently integrable using classical computing).
    """
    if len(values)<=1:
        return 0
    
    mid = len(values) // 2
    left = values[:mid]
    right = values[mid:]
    if(sum(values)>0):
        f = sum(left)/sum(values)
    else:
        f = 0
    theta = 2*math.acos(math.sqrt(f))
    #print("rotate i_th qubits " + str(qbit_no)+ " of " + str(theta) + " controlled by " + str(control) + ".")
    binary_list_c_state = [int(bit) for bit in control]
    if(control == ""):
        qc.ry(theta, qbit_no)
    else:
        cry_gate = controlled_rotation_on_n(theta, binary_list_c_state)
        qbits = []
        qbits.extend(list(range(qbit_no)))
        qbits.append(qbit_no)
        qbits.append(ancilla_no)
        qc.append(cry_gate, qbits)
    split_qc(qc, left, control + "0", qbit_no+1, ancilla_no)
    split_qc(qc, right, control + "1", qbit_no+1, ancilla_no)

def limited_split_qc(qc, values: list, control, qbit_no:int, ancilla_no:int, limit: int):
    """
    Create a circuit that recursivelly split up to a certain number of qubits.
    """
    if qbit_no > limit-1:
        return 0
    if len(values)<=1:
        return 0
    
    mid = len(values) // 2
    left = values[:mid]
    right = values[mid:]
    if(sum(values)>0):
        f = sum(left)/sum(values)
    else:
        f = 0
    theta = 2*math.acos(math.sqrt(f))
    #print("rotate i_th qubits " + str(qbit_no)+ " of " + str(theta) + " controlled by " + str(control) + ".")
    binary_list_c_state = [int(bit) for bit in control]
    if(control == ""):
        qc.ry(theta, qbit_no)
    else:
        cry_gate = controlled_rotation_on_n(theta, binary_list_c_state)
        qbits = []
        qbits.extend(list(range(qbit_no)))
        qbits.append(qbit_no)
        qbits.append(ancilla_no)
        qc.append(cry_gate, qbits)
    limited_split_qc(qc, left, control + "0", qbit_no+1, ancilla_no, limit)
    limited_split_qc(qc, right, control + "1", qbit_no+1, ancilla_no, limit)
    
def edit_string(s, i):
    return s[:(i)] + s[(i)+1:]

def split_string_into_chunks_with_parentheses(input_string, d):
    # Check if d is valid
    if d <= 0:
        return "Number of chunks must be greater than 0"
    
    # Calculate the size of each chunk
    chunk_size = len(input_string) // d
    
    # Create a new string by joining segments of size chunk_size with ', '
    result = ', '.join(input_string[i * chunk_size:(i + 1) * chunk_size] for i in range(d))
    
    # Add parentheses at the beginning and end
    return f"({result})"

def filter_dict_by_value(input_dict, n):
    """
    Removes all entries from input_dict where the value is smaller than n.

    Args:
    input_dict (dict): Dictionary with entries to be filtered.
    n (int or float): Threshold value.

    Returns:
    dict: A new dictionary with entries where values are >= n.
    """
    return {key: value for key, value in input_dict.items() if value >= n}

def convert_keys_to_decimal(input_dict):
    """
    Converts binary string keys (in the format '(binary1, binary2)') to decimal tuple keys 
    after reversing each binary string.

    Args:
    input_dict (dict): Dictionary with binary string keys.

    Returns:
    dict: A new dictionary with keys as tuples of decimal values.
    """
    decimal_dict = {}
    for key, value in input_dict.items():
        # Strip parentheses and split the binary values
        binary1, binary2 = key.strip("()").split(", ")
        decimal_key = f"{(int(binary1, 2), int(binary2, 2))}"
        decimal_dict[decimal_key] = value
    return decimal_dict


    
def run_and_plot(n_qubits, qc, shots=10000, discard = None, dimension = 1, only_relevant = False, relevant_value_min = 100):
    """
    (Designed for a 1-dim gaussian state init. circuit)
    Run the circuit and measure its output, using StatevectorSampler with given number of shots (default 10k)
    Plot the output of the circuit with reordering such that it looks like a nice gaussian
    """
    sampler = StatevectorSampler()
    observable = SparsePauliOp(['Z' * int(n_qubits)])
    # Transpile circuit
    pm = generate_preset_pass_manager(optimization_level=1)
    isa_circuit = pm.run(qc)
    # Run using sampler
    result = sampler.run([qc], shots=shots).result()
    # Access result data for PUB 0
    data_pub = result[0].data
    # Access bitstring for the classical register "meas"
    bitstrings = data_pub.meas.get_bitstrings()
    print(f"The number of bitstrings is: {len(bitstrings)}")
    # Get counts for the classical register "meas"
    counts = data_pub.meas.get_counts()
    if discard != None:
        rm_count = 0
        for e in discard:
            counts = {edit_string(key, e-rm_count): value for key, value in counts.items()}
            rm_count= rm_count +1 
   
    if dimension>1:
        counts = {split_string_into_chunks_with_parentheses(key, dimension): value for key, value in counts.items()}
        counts = dict(sorted(counts.items(), key=lambda item: item[1]))
    else:
        counts = dict(sorted(counts.items(), key=lambda item: item[0]))
    # Data to plot

    
    if only_relevant:
        counts = filter_dict_by_value(counts, relevant_value_min)

    #check also dim = 2 cause the utility are only design for such dict
    if dimension == 2:
        counts = convert_keys_to_decimal(counts)
    # Extract values for normalization
    values = list(counts.values())
    # Calculate the sum of all values
    total_sum = sum(values)
    # Normalize the data by the sum
    normalized_values = [value / total_sum for value in values]
    # Create the histogram
    plt.figure(figsize=(10, 6))
    plt.bar(counts.keys(), normalized_values, color='skyblue')
    # Add title and labels
    plt.title('Normalized Histogram of Data (Over Sum)')
    plt.xlabel('Labels')
    plt.ylabel('Normalized Values (Proportion of Total)')
    # Show grid for better readability
    plt.grid(axis='y')
    # Show the plot
    plt.tight_layout()
    plt.show()
    return counts

def run_measure_plot(qc, shots: int, dim, discard: list = None,  only_relevant = False, relevant_value_min = 100):
    """
    This takes a quantum circuit as input, runs it and measure with sampler for a given number of shots.
    Discard a list of qubits while printing the result (to remove ancilla in the output graph)
    """
    qc.measure_all()
    data = run_and_plot(qc.num_qubits, qc, shots=shots, discard = discard, dimension = dim,  only_relevant = only_relevant, relevant_value_min = relevant_value_min)
    return data
    
    

In [3]:

from mpl_toolkits.mplot3d import Axes3D


def plot_3d_distribution(data, fig_name=""):
    """
    Plot a 3D probability distribution.

    Parameters:
    data (dict): A dictionary where keys are 3D coordinates (tuples) and values are probabilities.
    """
    # Extract coordinates and probabilities
    x = []
    y = []
    z = []
    probs = []
    
    for (i, j, k), prob in data.items():
        x.append(i)
        y.append(j)
        z.append(k)
        probs.append(prob)

    # Normalize probabilities for coloring
    norm_probs = np.array(probs) / max(probs)

    # Create 3D scatter plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(x, y, z, c=norm_probs, cmap='viridis', s=50)

    # Add color bar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Probability')

    # Set labels
    ax.set_xlabel('z1')
    ax.set_ylabel('z2')
    ax.set_zlabel('z3')
    ax.set_title('3D Probability Distribution')
    
    if fig_name!="":
        plt.savefig(fig_name)
    # Show plot
    plt.show()
    
def plot_2d_distribution(data, fig_name=""):
    """
    Plot a heatmap for 2D probability distribution.

    Parameters:
    data (dict): A dictionary where keys are 2D coordinates (tuples) and values are probabilities.
    """
    # Extract x, y, and probabilities
    x = [coord[0] for coord in data.keys()]
    y = [coord[1] for coord in data.keys()]
    probs = list(data.values())

    # Create grid for heatmap
    x_unique = np.unique(x)
    y_unique = np.unique(y)

    heatmap = np.zeros((len(x_unique), len(y_unique)))

    # Fill heatmap data
    for (i, j), prob in data.items():
        xi = np.where(x_unique == i)[0][0]
        yi = np.where(y_unique == j)[0][0]
        heatmap[xi, yi] = prob

    # Plot heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, origin='lower', cmap='viridis', aspect='auto')
    plt.colorbar(label='Probability')

    # Label axes with reduced ticks for clarity
    plt.xticks(ticks=np.arange(0, len(y_unique), max(1, len(y_unique) // 10)), 
               labels=[y_unique[i]/len(y_unique) for i in range(0, len(y_unique), max(1, len(y_unique) // 10))])
    plt.yticks(ticks=np.arange(0, len(x_unique), max(1, len(x_unique) // 10)), 
               labels=[x_unique[i]/len(x_unique) for i in range(0, len(x_unique), max(1, len(x_unique) // 10))])
    
    plt.xlabel('Z_1')
    plt.ylabel('Z_2')
    plt.title('2D Probability Heatmap')
    
    if fig_name!="":
        plt.savefig(fig_name)
    # Show plot
    
    plt.show()

    
def generate_labels(probabilities, bit_size, dimension):
    """
    Generate labels as n integers from a given bit size and dimension.

    Parameters:
    probabilities (list): List of probabilities.
    bit_size (int): Size of each segment in bits.
    dimension (int): Number of segments to split the bit string into.

    Returns:
    dict: A dictionary mapping tuples of integers to probabilities.
    """
    # Total bit length
    total_bits = dimension * bit_size

    # Generate formatted probabilities
    if dimension>1:
        formatted_probs = {
            tuple(int(label[i * bit_size:(i + 1) * bit_size], 2) for i in range(dimension)): prob
            for label, prob in zip([f"{i:0{total_bits}b}" for i in range(len(probabilities))], probabilities)
            if prob > 1e-7
        }
    else:                            
        formatted_probs = {
            int(label,2): prob
            for label, prob in zip([f"{i:0{bit_size}b}" for i in range(len(probabilities))], probabilities)
            if prob > 1e-7
        }
    return formatted_probs